In [1]:
import json

In [2]:
j = '''
{
    "a": 100,
    "b": [1, 2, 3],
    "c": "python",
    "d": {
        "e": 4,
        "f": 5.5
    }
}
'''

In [3]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        print("decode: ", type(arg), arg)
        return "a simple string object"

In [5]:
result = json.loads(j, cls=CustomDecoder)

decode:  <class 'str'> 
{
    "a": 100,
    "b": [1, 2, 3],
    "c": "python",
    "d": {
        "e": 4,
        "f": 5.5
    }
}



In [6]:
result

'a simple string object'

In [7]:
class Point:
    def __init__(self, x, y):
        self.x = x,
        self.y = y,
        
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

In [9]:
j_points = '''
{
    "points": [
        [10, 20],
        [-1, -2],
        [0.5, 0.5]
    ]
}
'''

j_other = '''
{
    "a": 1,
    "b": 2
}
'''

In [15]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        if 'points' in arg:
            obj = json.loads(arg)
            return "parsing object for points"
        else:
            return json.loads(arg)

In [16]:
json.loads(j_points, cls=CustomDecoder)

'parsing object for points'

In [17]:
json.loads(j_other, cls=CustomDecoder)

{'a': 1, 'b': 2}

In [18]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        if 'points' in obj:
            obj['points'] = [Point(x, y)
                            for x, y in obj['points']]
        return obj

In [19]:
json.loads(j_points, cls=CustomDecoder)

{'points': [Point(x=(10,), y=(20,)),
  Point(x=(-1,), y=(-2,)),
  Point(x=(0.5,), y=(0.5,))]}

In [21]:
json.loads(j_other, cls=CustomDecoder)

{'a': 1, 'b': 2}

In [22]:
json.loads(j, cls=CustomDecoder)

{'a': 100, 'b': [1, 2, 3], 'c': 'python', 'd': {'e': 4, 'f': 5.5}}

```{"_type": "point", "x": x-coord, "y": y-coord}```

In [23]:
import re

In [24]:
pattern = r'"_type"\s*:\s*"point"'

r' = 'raw string'

Python won't interpret things like \t or \n as tabs or new lines. They'll simply print literally.

In [25]:
print('word\tword')

word	word


In [26]:
print(r'word\tword')

word\tword


In [27]:
regexp = re.compile(pattern)

In [28]:
print(regexp.search('"a": 1'))

None


In [31]:
print(regexp.search('"_type" :   \t "point"'))

<re.Match object; span=(0, 21), match='"_type" :   \t "point"'>


In [33]:
re.search(r'"_type"\s*:\s*"point"', '"_type" :   \t "point"')

<re.Match object; span=(0, 21), match='"_type" :   \t "point"'>

In [34]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj = self.make_pts(obj)
        return obj
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if obj.get('_type', None) == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
        return obj

In [58]:
j = '''
{
    "a": 100,
    "b": 0.5,
    "rectangle": {
        "corners": {
            "b_left": {"_type": "point", "x": -1, "y": -1},
            "b_right": {"_type": "point", "x": 1, "y": -1},
            "t_left": {"_type": "point", "x": -1, "y": 1},
            "t_right": {"_type": "point", "x": 1, "y": 1}
        },
        "rotate": {"_type" : "point", "x": 0, "y": 0},
        "interior_pts": [
            {"_type": "point", "x": 0, "y": 0},
            {"_type": "point", "x": 0.5, "y": 0.5}
        ]
    }
}
'''

In [62]:
def make_pts(obj):
    if isinstance(obj, dict):
        if obj.get('_type', None) == 'point':
            obj = Point(obj['x'], obj['y'])
        else:
            for key, value in obj.items():
                obj[key] = make_pts(value)
    elif isinstance(obj, list):
        for index, item in enumerate(obj):
            obj[index] = make_pts(item)
    return obj

In [69]:
make_pts({"_type": "point", "x": 1, "y": 1})

Point(x=(1,), y=(1,))

In [70]:
d = {"_type": "point", "x": 1, "y": 1}

In [75]:
Point(d['x'], d['y'])

Point(x=(1,), y=(1,))

In [67]:
p = Point(1, 1)

In [77]:
type(p.x)

tuple

In [78]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

In [79]:
json.loads(j)

{'a': 100,
 'b': 0.5,
 'rectangle': {'corners': {'b_left': {'_type': 'point', 'x': -1, 'y': -1},
   'b_right': {'_type': 'point', 'x': 1, 'y': -1},
   't_left': {'_type': 'point', 'x': -1, 'y': 1},
   't_right': {'_type': 'point', 'x': 1, 'y': 1}},
  'rotate': {'_type': 'point', 'x': 0, 'y': 0},
  'interior_pts': [{'_type': 'point', 'x': 0, 'y': 0},
   {'_type': 'point', 'x': 0.5, 'y': 0.5}]}}

In [80]:
from pprint import pprint
pprint(json.loads(j, cls=CustomDecoder))

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [81]:
from decimal import Decimal

In [82]:
CustomDecoder = json.JSONDecoder(parse_float=Decimal)

In [83]:
d = CustomDecoder.decode(j)

In [84]:
pprint(d)

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': {'_type': 'point', 'x': -1, 'y': -1},
                           'b_right': {'_type': 'point', 'x': 1, 'y': -1},
                           't_left': {'_type': 'point', 'x': -1, 'y': 1},
                           't_right': {'_type': 'point', 'x': 1, 'y': 1}},
               'interior_pts': [{'_type': 'point', 'x': 0, 'y': 0},
                                {'_type': 'point',
                                 'x': Decimal('0.5'),
                                 'y': Decimal('0.5')}],
               'rotate': {'_type': 'point', 'x': 0, 'y': 0}}}


In [85]:
class CustomDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        super().__init__(parse_float=Decimal)
    
    def decode(self, arg):
        obj = super().decode(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj = self.make_pts(obj)
        return obj
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if obj.get('_type', None) == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
        return obj

In [86]:
pprint(json.loads(j, cls=CustomDecoder))

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [91]:
result = json.loads(j, cls=CustomDecoder)

In [93]:
pprint(result)

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [94]:
pt = result['rectangle']['interior_pts'][1]
pprint(type(pt.y))

<class 'decimal.Decimal'>


In [89]:
type(pt.x)

decimal.Decimal

In [90]:
pt.x[1]

TypeError: 'decimal.Decimal' object is not subscriptable